In [1]:
import numpy as np
import scipy.linalg as la

In [10]:
import numpy as np
import scipy.linalg as la


#This function creates traingles from elements
def tet_to_tris(element):
    tris = [[None for j in range(3)]for i in range(4*len(element))]
    for i in range(len(element)):
        j = 4*i
        n0 = element[i][0]
        n1 = element[i][1]
        n2 = element[i][2]
        n3 = element[i][3]
        
        #Triangle-0
        tris[j][0] = n0
        tris[j][1] = n1
        tris[j][2] = n3
        
        #Triangle-1
        tris[j+1][0] = n1
        tris[j+1][1] = n2
        tris[j+1][2] = n3
        
        #Triangle-2
        tris[j+2][0] = n0
        tris[j+2][1] = n3
        tris[j+2][2] = n2
        
        #Triangle-3
        tris[j+3][0] = n0
        tris[j+3][1] = n2
        tris[j+3][2] = n1
      
    return tris    
with open('CONROD_NODE.txt','r') as f:
 node1 = np.array([[float(num) for num in line.split(',')] for line in f])
 
numnode = len(node1)
node = np.zeros((numnode,3))
for i in range(0,numnode):
    node[i,:] = node1[i,1:4]
print (node)
np.save('node',node)
#read element
with open('CONROD_ELEM.txt','r') as f:
 element = np.array([[int(num) for num in line.split(',')] for line in f])
#element = element.astype(int)
#print (element)

numelem = len(element)
#"element connectvity with zero base"
element2 = np.zeros((numelem,10))
for el in range(0,numelem):
 econ = element[el,1:11]
 element2[el,:] = econ-1
#print ("element connectvity with zero base")
#print (element2)

element = element2.astype('int')
#with open('spanner_Disp.txt','r') as f:
with open('CONROD_AMPLITUDE.txt','r') as f:
 stress = np.array([[float(num) for num in line.split()] for line in f])
          
 
Data = stress[:,1]
Data = Data.reshape(-1,1)
Stmax = max(Data)[0]  #Calculate the max stress
#=======================================================#
#max value - location
result = np.where(Data == np.amax(Data))
ind = result[0][0]
#====================================
values = Data.astype('str')  #To probe the value convert data as string
#=======================================================================
#Arrange triangles properly for plotting
tris = tet_to_tris(element)
numtri = len(tris)
idi = np.zeros(numtri)
idj = np.zeros(numtri)
idk = np.zeros(numtri)

for i in range(0,numtri):
    idi[i] = tris[i][0]
    idj[i] = tris[i][1]
    idk[i] = tris[i][2]
    
idi = idi.astype('int')
idj = idj.astype('int')
idk = idk.astype('int')

#================================================================
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'browser'

fig = go.Figure(data=[
    go.Mesh3d(
        x=node[:,0],
        y=node[:,1],
        z=node[:,2],
        colorbar_title='S,Mises',
        colorscale='jet',#'jet_r' to reverse map
        
        # Intensity of each vertex, which will be interpolated and color-coded
        intensity=Data,
        # i, j and k give the vertices of triangles
        # here we represent the 4 triangles of the tetrahedron surface
        i=idi,
        j=idj,
        k=idk,
        cmin = 0,
        cmax = Stmax,
        name='y',
        
        text = ["Value: "+j for j in values[:,0]],
       # showscale=False,
       opacity = 1,
     
    )
])
    
#To show the location of max stress - 3D Annotation
fig.update_layout(
    scene=dict(
        annotations=[
       
        dict(
            x=str(node[ind,0]),
            y=str(node[ind,1]),
            z=str(node[ind,2]),
            text="max: "+str(np.round(Data[ind,0],3)),
            textangle=0,
            ax=0,
            ay=-75,
            font=dict(
                color="black",
                size=12
            ),
            arrowcolor="black",
            arrowsize=3,
            arrowwidth=1,
            arrowhead=1),
        ]
    ),
)
#Write html file
fig.update_layout(scene=dict(
                 aspectmode='data'))
#fig.show()
fig.write_html('CONROD_SVM.html')
print('completed')


[[-75.34053341 -89.78735136 -31.90055641]
 [-74.67338424 -88.99227394 -34.34666033]
 [-86.5          0.         -39.6       ]
 ...
 [113.8383427   17.23863063 -39.6       ]
 [117.0431586   16.63217935 -35.64877387]
 [116.275949    11.9144612  -33.66521966]]
completed
